### **Importação das Bases de Dados**

In [ ]:
import pandas as pd
import os
from datetime import datetime
from disparador_email import enviar_email

data_atual = datetime.today().strftime('%d-%m-%Y')
def money_format(valor): 
    try:
        valor = f'R${int(valor):,.2f}'.replace(',', '#').replace('.', ',').replace('#', '.')
    except:
        valor = 'Sem Venda'

    return valor

df_emails = pd.read_excel("Bases de Dados/Emails.xlsx")
df_emails['E-mail'] = df_emails['E-mail'].str.removeprefix('pythonimpressionador+')
display(df_emails)
print(df_emails.info())

In [ ]:
df_lojas = pd.read_csv("Bases de Dados/Lojas.csv", sep=';', encoding='latin-1', dtype={'ID Loja': str})
display(df_lojas.head())
print(df_lojas.info())

In [ ]:
df_vendas = pd.read_excel("Bases de Dados/Vendas.xlsx", dtype={'ID Loja': str})
display(df_vendas.head())
print(df_vendas.info())

### **Para os Gerentes**

<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <style>    
    table {
      border-collapse: collapse;
      width: auto;
      margin-bottom: 16px;
    }
    th, td {
      border: 1px solid #999;
      padding: 4px 8px;
      text-align: center;
    }
    th {
      font-weight: bold;
    }
  </style>
</head>
<body>

<p>Bom dia, Tananana</p>

<p>O resultado de ontem foi:</p>

<table>
  <tr>
    <th>Indicador</th>
    <th>Valor Dia</th>
    <th>Meta Dia</th>
    <th>Cenário Dia</th>
  </tr>
  <tr>
    <td>Indicador 1</td>
    <td>x</td>
    <td>y</td>
    <td>Bom</td>
  </tr>
  <tr>
    <td>Indicador 2</td>
    <td>x</td>
    <td>y</td>
    <td>Ruim</td>
  </tr>
  <tr>
    <td>Indicador 3</td>
    <td>x</td>
    <td>z</td>
    <td>Bom</td>
  </tr>
</table>

<table>
  <tr>
    <th>Indicador</th>
    <th>Valor Ano</th>
    <th>Meta Ano</th>
    <th>Cenário Ano</th>
  </tr>
  <tr>
    <td>Indicador 1</td>
    <td>x</td>
    <td>y</td>
    <td>Bom</td>
  </tr>
  <tr>
    <td>Indicador 2</td>
    <td>x</td>
    <td>y</td>
    <td>Ruim</td>
  </tr>
  <tr>
    <td>Indicador 3</td>
    <td>x</td>
    <td>z</td>
    <td>Bom</td>
  </tr>
</table>

<p>Segue em anexo a planilha com todos os dados para mais detalhes.<br>
Qualquer dúvida, estou à disposição.</p>

<p>Att.,<br>
Fulano</p>

</body>
</html>

In [ ]:
ultimo_dia = max(df_vendas['Data'])

meta_faturamento_dia = 1000
meta_diversidade_produtos_vendidos_dia = 4
meta_ticket_medio_por_venda_dia = 50

meta_faturamento_ano = 1650000
meta_diversidade_produtos_vendidos_ano = 120
meta_ticket_medio_por_venda_ano = 500

with open(r'mensagens_email\mensagem-gerentes.html', 'r+', encoding='utf-8') as arquivo_html:
    corpo_email_gerentes = arquivo_html.read()

print(corpo_email_gerentes.count('{}')) 

In [ ]:
if not os.path.exists('BackupLojas - Vendas'):
    os.mkdir('BackupLojas - Vendas')

email = r'felipe.monsef.vasc@gmail.com'

for index in df_emails[:-1].index:
    nome_loja = df_emails.loc[index, 'Loja']
    nome_gerente = df_emails.loc[index, 'Gerente']
    # email_gerente = df_emails.loc[index, 'E-mail']

    filtro_loja = df_lojas['Loja'] == nome_loja
    id_loja = df_lojas[filtro_loja]['ID Loja'].tolist()[0]

    filtro_id_loja = df_vendas['ID Loja'] == id_loja
    df_export = df_vendas[filtro_id_loja].copy()


    # consolidado do dia
    filtro_dia = df_export['Data'] == ultimo_dia
    df_vendas_ultimo_dia = df_export[filtro_dia].copy()
    
    faturamento_total_dia = df_vendas_ultimo_dia['Valor Final'].sum()
    qtd_produtos_vendidos_dia = len(df_vendas_ultimo_dia['Produto'].unique())
    ticket_medio_por_venda_dia = df_vendas_ultimo_dia.groupby('Código Venda')[['Valor Final']].mean().reset_index()['Valor Final'].mean()


    # consolidado do ano
    faturamento_total_ano = df_export['Valor Final'].sum()
    qtd_produtos_vendidos_ano = len(df_export['Produto'].unique())
    ticket_medio_por_venda_ano = df_export.groupby('Código Venda')[['Valor Final']].mean().reset_index()['Valor Final'].mean()


    if faturamento_total_dia >= meta_faturamento_dia:
        cenario_faturamento_dia = '✅'
    else:
        cenario_faturamento_dia = '❌'

    if qtd_produtos_vendidos_dia >= meta_diversidade_produtos_vendidos_dia:
        cenario_produtos_dia = '✅'
    else:
        cenario_produtos_dia = '❌'

    if ticket_medio_por_venda_dia >= meta_ticket_medio_por_venda_dia:
        cenario_ticket_medio_dia = '✅'
    else:
        cenario_ticket_medio_dia = '❌'


    if faturamento_total_ano >= meta_faturamento_ano:
        cenario_faturamento_ano = '✅'
    else:
        cenario_faturamento_ano = '❌'

    if qtd_produtos_vendidos_ano >= meta_diversidade_produtos_vendidos_ano:
        cenario_produtos_ano = '✅'
    else:
        cenario_produtos_ano = '❌'

    if ticket_medio_por_venda_ano >= meta_ticket_medio_por_venda_ano:
        cenario_ticket_medio_ano = '✅'
    else:
        cenario_ticket_medio_ano = '❌'



    email_enviar = corpo_email_gerentes.format(
        nome_gerente, ultimo_dia.strftime("%d/%m/%Y"), nome_loja,
        money_format(faturamento_total_dia), money_format(meta_faturamento_dia), cenario_faturamento_dia, 
        qtd_produtos_vendidos_dia, meta_diversidade_produtos_vendidos_dia, cenario_produtos_dia,
        money_format(ticket_medio_por_venda_dia), money_format(meta_ticket_medio_por_venda_dia), cenario_ticket_medio_dia,
        money_format(faturamento_total_ano), money_format(meta_faturamento_ano), cenario_faturamento_ano, 
        qtd_produtos_vendidos_ano, meta_diversidade_produtos_vendidos_ano, cenario_produtos_ano,
        money_format(ticket_medio_por_venda_ano), money_format(meta_ticket_medio_por_venda_ano), cenario_ticket_medio_ano,
        'Felipe',
    )

    assunto = f'OnePage Dia {ultimo_dia.strftime("%d-%m-%Y")} - {nome_loja}'


    df_export['Valor Final'] = df_export['Valor Final'].astype(str)
    df_export.loc[:, 'Valor Final'] = df_export['Valor Final'].apply(money_format)

    if not os.path.exists(rf'BackupLojas - Vendas\Vendas - {nome_loja}'):
        os.mkdir(rf'BackupLojas - Vendas\Vendas - {nome_loja}')

    if not os.path.exists(rf'BackupLojas - Vendas\Vendas - {nome_loja}\{ultimo_dia} - {nome_loja}.xlsx'):
        df_export.to_excel(rf'BackupLojas - Vendas\Vendas - {nome_loja}\{ultimo_dia.strftime("%d-%m-%Y")} - {nome_loja}.xlsx', index=False)
    
    enviar_email(assunto, email_enviar, rf'BackupLojas - Vendas\Vendas - {nome_loja}\{ultimo_dia.strftime("%d-%m-%Y")} - {nome_loja}.xlsx', f'{ultimo_dia.strftime("%d-%m-%Y")} - {nome_loja}.xlsx')

    break


### **Para a Diretoria**

<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Ranking de Lojas</title>
</head>
<body style="font-family: Arial, sans-serif; line-height: 1.6;">

  <p>Prezada Diretoria,</p>

  <p>Encaminho em anexo os rankings de desempenho das lojas, conforme segue:</p>
  <ul>
    <li><strong>Ranking diário</strong> – referente ao dia <strong>{}</strong>;</li>
    <li><strong>Ranking anual</strong> – consolidado até <strong>{}</strong>.</li>
  </ul>

  <p><strong>Destaques do dia:</strong></p>
  <ul>
    <li>🏆 <strong>Melhor loja do dia:</strong> {}</li>
    <li>⚠️ <strong>Pior desempenho do dia:</strong> {}</li>
  </ul>

  <p><strong>Acumulado do ano:</strong></p>
  <ul>
    <li>🥇 <strong>Melhor loja do ano:</strong> {}</li>
    <li>🔻 <strong>Pior desempenho do ano:</strong> {}</li>
  </ul>

  <p>Fico à disposição para qualquer esclarecimento.</p>

  <p>Att.,<br>
  {}<br>
</body>
</html>

In [ ]:
with open(r'mensagens_email\mensagem-diretoria.html', 'r+', encoding='utf-8') as arquivo_html:
    corpo_email_diretoria = arquivo_html.read()

print(corpo_email_diretoria.count('{}')) 

In [ ]:
df_ranking_lojas_ano = df_vendas.groupby('ID Loja')[['Valor Final']].sum().reset_index().sort_values(by='Valor Final', ascending=False)
df_ranking_lojas_ano = pd.merge(df_ranking_lojas_ano, df_lojas, on='ID Loja')
df_ranking_lojas_ano['Valor Final'] = df_ranking_lojas_ano['Valor Final'].apply(money_format)
df_ranking_lojas_ano['Rankgin'] = [f'Top {index}' for index in range(1, len(df_ranking_lojas_ano['Loja']) + 1)]

melhor_loja_ano = df_ranking_lojas_ano['Loja'].tolist()[0]
pior_loja_ano = df_ranking_lojas_ano['Loja'].tolist()[-1]

print(melhor_loja_ano)
print(pior_loja_ano)
# -------------------------------------------

df_vendas_ultimo_dia = df_vendas[(df_vendas['Data'] == ultimo_dia)]
df_ranking_lojas_dia = pd.merge(df_vendas_ultimo_dia.groupby('ID Loja')[['Valor Final']].sum().reset_index().sort_values(by='Valor Final', ascending=False), df_lojas, on='ID Loja')
df_ranking_lojas_dia['Valor Final'] = df_ranking_lojas_dia['Valor Final'].apply(money_format)
df_ranking_lojas_dia['Rankgin'] = [f'Top {index}' for index in range(1, len(df_ranking_lojas_dia['Loja']) + 1)]

melhor_loja_dia = df_ranking_lojas_dia['Loja'].tolist()[0]
pior_loja_dia = df_ranking_lojas_dia['Loja'].tolist()[-1]

print(melhor_loja_dia)
print(pior_loja_dia)


if not os.path.exists(r'BackupLojas - Vendas\Vendas - Diretoria'):
    os.mkdir(r'BackupLojas - Vendas\Vendas - Diretoria')

arquivo_dia = rf'BackupLojas - Vendas\Vendas - Diretoria\RankingLojasDia - {ultimo_dia.strftime("%d-%m-%Y")}.xlsx'
arquivo_ano = rf'BackupLojas - Vendas\Vendas - Diretoria\RankingLojasAno - até {ultimo_dia.strftime("%d-%m-%Y")}.xlsx'

if ((not os.path.exists(arquivo_dia)) and (not os.path.exists(arquivo_ano))):
    df_ranking_lojas_ano.to_excel(arquivo_dia, index=False)
    df_ranking_lojas_dia.to_excel(arquivo_ano, index=False)

corpo_email_diretoria = corpo_email_diretoria.format(
    ultimo_dia.strftime('%d/%m/%Y'), 
    ultimo_dia.strftime('%d/%m/%Y'),
    melhor_loja_dia, pior_loja_dia,
    melhor_loja_ano, pior_loja_ano,
    'Felipe'
)

enviar_email(
    f'Rankings de desempenho das lojas - {ultimo_dia.strftime("%d/%m/%Y")}', 
    corpo_email_diretoria,
    (arquivo_dia, arquivo_ano), (f'RankingLojasDia - {data_atual}.xlsx', f'RankingLojasAno - {data_atual}.xlsx'),
     qtd_anexos=2
)